# Manejo de Valores Nulos - Telecom X

## Propósito
En este cuaderno, se abordan los 11 valores nulos en la columna `account_Charges.Total` identificados previamente. Se imputan estos valores usando una estrategia basada en los cargos mensuales y la duración del contrato, y se verifica la coherencia del dataset final.

## Tarea
1. Cargar el dataset desde la URL de la API y aplicar las correcciones previas.
2. Analizar las filas con valores nulos en `account_Charges.Total`.
3. Imputar los valores nulos usando una estrategia adecuada.
4. Verificar la coherencia del dataset tras la imputación.
5. Guardar el dataset final limpio.

In [1]:
# Importación de bibliotecas
import pandas as pd
import requests
import json

# Carga de datos desde la API
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
response = requests.get(url)
data = response.json()

# Conversión a DataFrame
df = pd.DataFrame(data)

# Desanidar columnas JSON
customer_df = pd.json_normalize(df['customer'])
customer_df.columns = [f"customer_{col}" for col in customer_df.columns]
phone_df = pd.json_normalize(df['phone'])
phone_df.columns = [f"phone_{col}" for col in phone_df.columns]
internet_df = pd.json_normalize(df['internet'])
internet_df.columns = [f"internet_{col}" for col in internet_df.columns]
account_df = pd.json_normalize(df['account'])
account_df.columns = [f"account_{col}" for col in account_df.columns]

# Combinar los datos desanidados
df_clean = pd.concat([df[['customerID', 'Churn']], customer_df, phone_df, internet_df, account_df], axis=1)

# Aplicar correcciones previas
df_clean['Churn'] = df_clean['Churn'].replace('', 'No')
df_clean['account_Charges.Monthly'] = pd.to_numeric(df_clean['account_Charges.Monthly'], errors='coerce')
df_clean['account_Charges.Total'] = pd.to_numeric(df_clean['account_Charges.Total'], errors='coerce')
df_clean['customer_tenure'] = df_clean['customer_tenure'].astype('int64')
df_clean['customer_SeniorCitizen'] = df_clean['customer_SeniorCitizen'].astype('int64')

# 1. Análisis de filas con valores nulos en 'account_Charges.Total'
print("Filas con valores nulos en 'account_Charges.Total':")
null_rows = df_clean[df_clean['account_Charges.Total'].isnull()]
print(null_rows[['customerID', 'customer_tenure', 'account_Charges.Monthly', 'account_Charges.Total']])

# 2. Imputación de valores nulos
# Si 'customer_tenure' es 0 o 1, asumimos que 'account_Charges.Total' es igual a 'account_Charges.Monthly'
# Para otros casos, imputamos con la mediana de 'account_Charges.Total' para evitar sesgos
median_total = df_clean['account_Charges.Total'].median()
df_clean['account_Charges.Total'] = df_clean.apply(
    lambda row: row['account_Charges.Monthly'] if pd.isna(row['account_Charges.Total']) and row['customer_tenure'] <= 1
    else row['account_Charges.Total'] if not pd.isna(row['account_Charges.Total'])
    else median_total,
    axis=1
)

# 3. Verificación de valores nulos tras imputación
print("\nValores nulos tras imputación:")
print(df_clean.isnull().sum())

# 4. Verificación de coherencia en variables numéricas
numeric_columns = ['customer_tenure', 'account_Charges.Monthly', 'account_Charges.Total']
print("\nEstadísticas descriptivas de variables numéricas tras imputación:")
print(df_clean[numeric_columns].describe())

# Verificar valores negativos
for col in numeric_columns:
    if (df_clean[col] < 0).any():
        print(f"\nValores negativos encontrados en {col}:")
        print(df_clean[df_clean[col] < 0][col])
    else:
        print(f"\nNo se encontraron valores negativos en {col}.")

# 5. Verificación final
print("\nTipos de datos después de la imputación:")
print(df_clean.dtypes)
print("\nPrimeras 5 filas del DataFrame corregido:")
print(df_clean.head())

# 6. Guardar el dataset final limpio
df_clean.to_csv('TelecomX_Data_final.csv', index=False)
print("\nDataset final limpio guardado como 'TelecomX_Data_final.csv'.")

Filas con valores nulos en 'account_Charges.Total':
      customerID  customer_tenure  account_Charges.Monthly  \
975   1371-DWPAZ                0                    56.05   
1775  2520-SGTTA                0                    20.00   
1955  2775-SEFEE                0                    61.90   
2075  2923-ARZLG                0                    19.70   
2232  3115-CZMZD                0                    20.25   
2308  3213-VVOLG                0                    25.35   
2930  4075-WKNIU                0                    73.35   
3134  4367-NUYAO                0                    25.75   
3203  4472-LVYGI                0                    52.55   
4169  5709-LVOEQ                0                    80.85   
5599  7644-OMVMY                0                    19.85   

      account_Charges.Total  
975                     NaN  
1775                    NaN  
1955                    NaN  
2075                    NaN  
2232                    NaN  
2308                   

## Descripción de la tarea
En esta sección, se analizaron las 11 filas con valores nulos en `account_Charges.Total`. Los valores nulos se imputaron usando `account_Charges.Monthly` para clientes con `customer_tenure` menor o igual a 1, y la mediana de `account_Charges.Total` para otros casos. Se verificó que no queden valores nulos ni negativos, y que los tipos de datos sean correctos. El dataset final limpio se guardó como `TelecomX_Data_final.csv` para su uso en el análisis exploratorio.